# Relational Databases

In the following it is assumed that the reader is familar with the concept of a relational database and the SQL syntax.

In [26]:
import sqlalchemy

## SQLite

SQLite is a light-weight relational database where the data is stored either in memory or in a single file. In contrast to most other database technologies, it does not have a client/ server architecture.

SQLite is very commonly used (e.g. for Android), a (potentially a bit biassed) discussion of the use cases is given here:

https://www.sqlite.org/whentouse.html

The easiest way to use SQLite in Python would be the *sqlite3* standard library. Here however we are using the SQLAlchemy engine, which yields a better portability to other database technologies and allows usage of the SQLAlchemy ORM (see below).

In [29]:
# engine = sqlalchemy.create_engine('sqlite:///test.db') # database in file
engine = sqlalchemy.create_engine('sqlite:///:memory:') # in-memory database
con = engine.connect()

In [30]:
table_name = 'test1'

In [31]:
sql = f"""
CREATE TABLE {table_name} (
Id Int,
Name Varchar,
Amount Numeric(10,2))
"""
con.execute(sql)

In [32]:
sql = f"""
INSERT INTO {table_name} (Id, Name, Amount) 
VALUES 
(1, 'hello', 42),
(2, 'world', 7.42),
(3, '!', -1)
"""
con.execute(sql)

In [33]:
sql = f"""
select Id, Name
from {table_name}
where Amount > 0
"""
result = con.execute(sql)
for line in result:
    print(line)

(1, 'hello')
(2, 'world')


In [34]:
sql = f"""
DROP TABLE {table_name}
"""
con.execute(sql)

## Postgres

This tutorial requires a running Postgres server, preferably in a Docker container started using the given docker-compose file.

In [1]:
!conda install psycopg2 --yes # installs driver for Postgres

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::matplotlib==3.0.3=py37_1
done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - psycopg2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libpq-11.4                 |       hd9ab2ff_1         2.4 MB  conda-forge
    psycopg2-2.8.3             |   py37h72c5cf5_0         165 KB  conda-forge
    pyqt-5.9.2                 |   py37hcca6a23_0         5.7 MB  conda-forge
    qt-5.9.7                   |       h52cfd70_2        85.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        94.2 MB

The following NEW packages will be INSTALLED:

  libpq              conda-forge/linux-64::libpq-11.4-hd9ab

In [71]:
username = 'postgres'
password = 'J8IzgZj3iFhYClDl' # 'python_tutorial_5432'
db_url = 'postgres_db:5432' # using Docker url alias
db_schema = 'postgres'
con_str = f'postgres://{username}:{password}@{db_url}/{db_schema}'

In [72]:
engine_pg = sqlalchemy.create_engine(con_str)
con_pg = engine_pg.connect()

In [10]:
table_name = 'test1'

In [15]:
sql = f"""
CREATE TABLE {table_name} (
Id Int,
Name Varchar,
Amount Numeric(10,2))
"""
con_pg.execute(sql)

In [16]:
sql = f"""
INSERT INTO {table_name} (Id, Name, Amount) 
VALUES 
(1, 'hello', 42),
(2, 'world', 7.42),
(3, '!', -1)
"""
con_pg.execute(sql)

In [17]:
sql = f"""
select Id, Name
from {table_name}
where Amount > 0
"""
result = con_pg.execute(sql)
for line in result:
    print(line)

(1, 'hello')
(2, 'world')


<a id='pandas'></a>
## Pandas Interaction

### Load Pandas Example Data

The example data of the [Pandas](pandas.ipynb) tutorial is used here, the download commands are in that tutorial.

In [79]:
import pandas as pd

In [80]:
test_csv = '../code/test_data.csv'
table_name = 'test_taxi'

In [81]:
df = pd.read_csv(test_csv, parse_dates=[1, 2], 
                 nrows=10000) # read only the first 10000 rows of data set
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
0,2,2018-01-01 00:18:50,2018-01-01 00:24:39,N,1,236,236,5,0.70,6.0,0.5,0.5,0.0,0.0,NaN,0.3,7.3,2,1
1,2,2018-01-01 00:30:26,2018-01-01 00:46:42,N,1,43,42,5,3.50,14.5,0.5,0.5,0.0,0.0,NaN,0.3,15.8,2,1
2,2,2018-01-01 00:07:25,2018-01-01 00:19:45,N,1,74,152,1,2.14,10.0,0.5,0.5,0.0,0.0,NaN,0.3,11.3,2,1
3,2,2018-01-01 00:32:40,2018-01-01 00:33:41,N,1,255,255,1,0.03,-3.0,-0.5,-0.5,0.0,0.0,NaN,-0.3,-4.3,3,1
4,2,2018-01-01 00:32:40,2018-01-01 00:33:41,N,1,255,255,1,0.03,3.0,0.5,0.5,0.0,0.0,NaN,0.3,4.3,2,1


### Write DataFrame to Table

In [82]:
engine = sqlalchemy.create_engine('sqlite:///:memory:') # in-memory database
con = engine.connect()

In [83]:
df.to_sql(table_name, # target table
          engine, # SQLAlchemy engine instance
          index=False, # no not write index to table - set this parameter according to your needs
          )

### Retrieve Data from Database

In [84]:
df_from_db = pd.read_sql_table(table_name, engine)
df_from_db.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
0,2,2018-01-01 00:18:50,2018-01-01 00:24:39,N,1,236,236,5,0.70,6.0,0.5,0.5,0.0,0.0,NaN,0.3,7.3,2,1
1,2,2018-01-01 00:30:26,2018-01-01 00:46:42,N,1,43,42,5,3.50,14.5,0.5,0.5,0.0,0.0,NaN,0.3,15.8,2,1
2,2,2018-01-01 00:07:25,2018-01-01 00:19:45,N,1,74,152,1,2.14,10.0,0.5,0.5,0.0,0.0,NaN,0.3,11.3,2,1
3,2,2018-01-01 00:32:40,2018-01-01 00:33:41,N,1,255,255,1,0.03,-3.0,-0.5,-0.5,0.0,0.0,NaN,-0.3,-4.3,3,1
4,2,2018-01-01 00:32:40,2018-01-01 00:33:41,N,1,255,255,1,0.03,3.0,0.5,0.5,0.0,0.0,NaN,0.3,4.3,2,1


The command *read_sql_table* loads a complete table from database into a Pandas DataFrame.

In [85]:
sql = f"""
select trip_distance, total_amount, passenger_count
from {table_name}
where VendorID = 2 and total_amount > 0
"""
df_from_db2 = pd.read_sql(sql, engine)
df_from_db2.head()

,trip_distance,total_amount,passenger_count
0,0.70,7.3,5
1,3.50,15.8,5
2,2.14,11.3,1
3,0.03,4.3,1
4,5.63,22.3,1


Execute an SQL Query on the database and load the result into a DataFrame. 

In [86]:
sql = f"""
select VendorID, payment_type, sum(trip_distance) as miles_total, 
sum(total_amount) as earnings_total, 
sum(passenger_count) as passengers_total
from {table_name}
where trip_distance > 0 and total_amount > 0
group by VendorID, payment_type
"""
df_from_db3 = pd.read_sql(sql, engine)
df_from_db3.head()

,VendorID,payment_type,miles_total,earnings_total,passengers_total
0,1,1,2745.10,13940.07,976
1,1,2,2086.40,9775.48,1014
2,1,3,96.00,434.50,34
3,1,4,18.60,101.97,8
4,2,1,13320.22,68793.66,5517


When working with large data sets, especially with a client-server database, it is recommended to filter and aggregate as much as possible already on database side using SQL.
This saves both memory for data storage in Pandas and IO (and potentially network) traffic between the database server and the analysis machine.

### Writing into Existing Table

In [87]:
try:
    df.to_sql(table_name, # target table
          engine, # SQLAlchemy engine instance
          )
except ValueError as e:
    print(e)

Table 'test_taxi' already exists.


In standard configuration, Pandas raises a *ValueError* if the target table exists.

In [88]:
df.to_sql(table_name, # target table
          engine, # SQLAlchemy engine instance
          index=False, # no not write index to table - set this parameter according to your needs
          if_exists='append', # if the table already exists, append entries in this df
         )

In [89]:
con.execute(f'select count(*) from {table_name}').fetchone()

(20000,)

The keyword argument *if_exists='append'* appends __all__ entries in the DataFrame to the database table.

Note that there is no check if rows in the DataFrame already exist in the table, this could lead to duplicates (like in the example above: 2000 rows in total, 2 times the same 1000 rows have been inserted).

In [90]:
df.to_sql(table_name, # target table
          engine, # SQLAlchemy engine instance
          index=False, # no not write index to table - set this parameter according to your needs
          if_exists='replace', # if the table already exists, delete the table and insert new data
          method='multi',
         )

In [91]:
con.execute(f'select count(*) from {table_name}').fetchone()

(10000,)

The keyword argument *if_exists='replace'* drops the database table with the given name first, then it creates a new table containing the data of the DataFrame.

Use this option with caution to avoid data losses.

Note that there is no built-in possibility in Pandas to insert only "new" data from a DataFrame into a database table. If this is requried, the corresponding logic must be written by the user.

### Performance 
#### SQLite

In [102]:
engine = sqlalchemy.create_engine('sqlite:///test.db') # database in file
con = engine.connect()

In [104]:
%%timeit
df.to_sql(table_name, # target table
          engine, # SQLAlchemy engine instance
          index=False, # no not write index to table - set this parameter according to your needs
          if_exists='replace', # if the table already exists, delete the table and insert new data
         )

1.3 s ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [105]:
%timeit df_from_db = pd.read_sql_table(table_name, engine)

190 ms ± 230 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Postgres

In [106]:
engine_pg = sqlalchemy.create_engine(con_str)
con_pg = engine_pg.connect()

In [108]:
%%timeit
df.to_sql(table_name, # target table
          engine_pg, # SQLAlchemy engine instance
          index=False, # no not write index to table - set this parameter according to your needs
          if_exists='replace', # if the table already exists, delete the table and insert new data
         )

5.85 s ± 56.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In some cases, setting the keyword argument *method='multi'* gives a better performance (but not in this example on my machine).

In [109]:
%timeit df_from_db = pd.read_sql_table(table_name, engine_pg)

278 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Cleanup

In [110]:
sql = f"""
DROP TABLE {table_name}
"""
con.execute(sql)

In [111]:
sql = f"""
DROP TABLE {table_name}
"""
con_pg.execute(sql)

In [112]:
%rm test.db

## Conclusion Pandas Database Integration

It is easy and rather performat to retrieve data from database tables into Pandas DataFrames. Using SQL, the requests can be written such that filtering and aggregation is already done on database server side, minimizing traffic and memory requirements.

Inserting data from Pandas Dataframes into database tables is also easily possible, but with limitations:

* There is no built-in way to insert only new or updated rows. It is only possible to insert complete DataFrames (with and without removing the existing table).
* Inserting large amounts of data can take long time.